In [8]:
import numpy as np
import matplotlib.pyplot as plt
import qutip as qt
import cmath
import statsmodels.api as sm
import scipy as sp

In [ ]:
a_1 = qt.tensor(qt.destroy(2), qt.qeye(2))
a_2 = qt.tensor(qt.qeye(2), qt.destroy(2))
b_1 = qt.destroy(2)
N_1 = a_1.dag()*a_1
N_2 = a_2.dag()*a_2
psi = qt.fock(2,0)
fi = qt.fock(2,1)

In [2]:
'''
This function calculates the Leggett-Garg factor averaging over an enesemble of trajectories. 
This is the object to confront with the theoretical prediction. 
See notebooks LG.ipynb, LG_feed.ipynb
'''
def legg_ensemble(ensemble):
    num_realizations, num_time_steps = ensemble.shape
    max_tau = num_time_steps // 2 

    differences = np.zeros(max_tau)

    for tau in range(1, max_tau):
        correlation_tau = np.mean(ensemble[:, 0] * ensemble[:, tau])
        correlation_2tau = np.mean(ensemble[:, 0] * ensemble[:, 2 * tau])
        differences[tau - 1] = 2*correlation_tau - correlation_2tau

    return differences


In [10]:
'''
This function calculates the Leggett-Garg factor as a time average. 
It is interesting to notive that without feedback it coincides with the ensemble one suggesting 
that the system possesses a certain level of ergodicity. When feedback is implemented, 
it starts to deviate from the ensemble one i. e. from the theoretical prediction.  
'''
def leggett_garg_factor(I_t):

    N = len(I_t)
    Legg = []
    K_tau_list = []
    K_2tau_list = []
    for tau in range(1, N // 2 + 1):  
        K_tau = np.mean(I_t[:-tau] * I_t[tau:])
        K_2tau = np.mean(I_t[:-2*tau] * I_t[2*tau:])  
        Legg.append(2*K_tau - K_2tau)
        K_tau_list.append(K_tau)
        K_2tau_list.append(K_2tau)

    return np.array(Legg), np.array(K_tau_list), np.array(K_2tau_list)

In [11]:
'''
Does the same thing as leggett_garg_factor(I_t) but for a shorter time window
'''
def leggett_garg_factor_lessteps(I_t):

    N = len(I_t)
    Legg = []
    K_tau_list = []
    K_2tau_list = []
    for tau in range(1, 800):  
        K_tau = np.mean(I_t[:-tau] * I_t[tau:])
        K_2tau = np.mean(I_t[:-2*tau] * I_t[2*tau:])
        Legg.append(2*K_tau - K_2tau)
        K_tau_list.append(K_tau)
        K_2tau_list.append(K_2tau)

    return np.array(Legg), np.array(K_tau_list), np.array(K_2tau_list)

In [12]:
def reshape_measurement(N_traj, N_times, measurement):
    ensemble = np.zeros((N_traj, N_times))
    for i in range(N_traj):
        for j in range(N_times):
            ensemble[i,j] = np.real(measurement[i][j][0]) 
    return ensemble

In [13]:
def average(array):
    new_array = []
    for i in range(len(arrays)):
        if len(arrays[i]) != 0:
            new_array.append(arrays[i])
    return np.mean(new_array, axis=0)

In [14]:
'''Plots the leggett garg factor comparing it with theoretical one'''

def plot_legg(times, av):
    
    chi = 0.1
    omega = 0.3
    dt = 0.1
    gamma = chi**2/2
    Tilde_om = 2*np.sqrt(omega**2 - gamma**2/16)
    K_t = 2*gamma*np.e**(-gamma/2*times)*(np.cos(Tilde_om*times) + gamma/(2*Tilde_om)*np.sin(Tilde_om*times))
    K_2t = 2*gamma*np.e**(-gamma/2*2*times)*(np.cos(Tilde_om*2*times) + gamma/(2*Tilde_om)*np.sin(Tilde_om*2*times))
    L = 2*K_t - K_2t
    
    
    fig, ax = plt.subplots()
    ax.plot(times[:len(av)], av, label='Simulation')
    ax.plot(times[:len(av)], L[:len(av)], label='Analytical result')
    ax.axhline(y=0.01, color='red', linestyle='--', label='LG limit')
    ax.legend()
    #ax.set_xlim(0,80)
    ax.set_title('Leggett-Garg factor QDD continuously monitored')
    ax.set_xlabel('Time')
    ax.set_ylabel('Value')

In [15]:
'''Plots the leggett garg factor for the feedback dynamics comparing it with theoretical one'''

def plot_legg_feed(times, av, mu):
    
    chi = 0.1
    omega = 0.3
    T = 10.
    zin = 1
    yin = 0
    B = 2*omega+2*T*mu+2*chi*mu
    delta = 4*B**2-chi**4/4
    alpha = 2*mu**2+1/4*chi**2
    Cost1 = 2*mu*chi*B/(B**2+mu**2*chi**2+4*mu**4)+zin
    Cost2 = 1/np.sqrt(delta)*(1/2*chi**2*zin+2*B*yin+2*mu*chi*B*(4*mu**2+1/4*chi**2)/(B**2+mu**2*chi**2+4*mu**4))
    Cost3 = 2*mu*chi*B/(B**2+mu**2*chi**2+4*mu**4)
    z = np.exp(-alpha*times)*(Cost1*np.cos(1/2*np.sqrt(delta)*times)+Cost2*np.sin(1/2*np.sqrt(delta)*times))-Cost3
    z2t = np.exp(-alpha*2*times)*(Cost1*np.cos(1/2*np.sqrt(delta)*2*times)+Cost2*np.sin(1/2*np.sqrt(delta)*2*times))-Cost3

    newyin=-2*zin*mu-yin*zin*chi
    newzin = 2*yin*mu+(1-zin**2)*chi
    newCost1 = 2*mu*chi*B/(B**2+mu**2*chi**2+4*mu**4)+newzin 
    newCost2 = 1/np.sqrt(delta)*(1/2*chi**2*newzin+2*B*newyin+2*mu*chi*B*(4*mu**2+1/4*chi**2)/(B**2+mu**2*chi**2+4*mu**4))
    
    znew = np.exp(-alpha*times)*(newCost1*np.cos(1/2*np.sqrt(delta)*times)+newCost2*np.sin(1/2*np.sqrt(delta)*times))-Cost3
    znew2t = np.exp(-alpha*2*times)*(newCost1*np.cos(1/2*np.sqrt(delta)*2*times)+newCost2*np.sin(1/2*np.sqrt(delta)*2*times))-Cost3
    FeedCorr = chi**2*z*zin+chi*znew
    FeedCorr2t = chi**2*z2t*zin+chi*znew2t
    LeggettGarg = 2*FeedCorr-FeedCorr2t
    
    fig, ax = plt.subplots()
    ax.plot(times[:len(av)], av, label='Simulation')
    ax.axhline(y=0.01, color='red', linestyle='--', label='LG limit')
    ax.plot(times[:len(LeggettGarg)], LeggettGarg, label='Feedback analytical solution')
    ax.legend()
    ax.set_xlim(0,15)
    ax.set_title('Leggett-Garg factor QDD continuously monitored feedback')
    ax.set_xlabel('Time')
    ax.set_ylabel('Value')
    
    